# FreqAI GPU Backtest - Remote Setup (Cloudflared)

✅ **100% FREE** - No credit card needed!

**Steps:**
1. Enable GPU: Runtime > Change runtime type > GPU (T4)
2. Run all cells: Runtime > Run all
3. Copy connection details
4. Run backtest from your local machine

In [ ]:
# 1. Check GPU
!nvidia-smi --query-gpu=name,memory.total --format=csv || echo '❌ GPU not found. Enable GPU!'

In [ ]:
%%capture
# 2. Install SSH
!apt-get update -qq
!apt-get install -y openssh-server

In [ ]:
# 3. Setup SSH
import secrets, string
password = ''.join(secrets.choice(string.ascii_letters + string.digits) for _ in range(16))
!mkdir -p /var/run/sshd
!echo "root:{password}" | chpasswd
!sed -i 's/#PermitRootLogin prohibit-password/PermitRootLogin yes/' /etc/ssh/sshd_config
!sed -i 's/#PasswordAuthentication yes/PasswordAuthentication yes/' /etc/ssh/sshd_config
!service ssh restart
print(f'✅ SSH ready | Password: {password}')

In [ ]:
# 4. Install Cloudflared (100% FREE!)
import os
os.chdir('/content')
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 cloudflared
!./cloudflared --version
print('\n✅ Cloudflared ready (no auth token needed!)')

In [ ]:
# 5. Start tunnel
import subprocess, time, re
import os
os.chdir('/content')
print('⏳ Starting Cloudflare Tunnel...')

# Start cloudflared
proc = subprocess.Popen(
    ['./cloudflared', 'tunnel', '--url', 'tcp://localhost:22'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True
)

# Wait for tunnel URL in output
tunnel_url = None
for _ in range(30):
    line = proc.stdout.readline()
    if not line:
        time.sleep(1)
        continue
    
    # Look for URL in output
    match = re.search(r'https://[a-z0-9-]+\.trycloudflare\.com', line)
    if match:
        tunnel_url = match.group(0)
        break

if tunnel_url:
    print('\n' + '='*70)
    print('✅ TUNNEL ACTIVE')
    print('='*70)
    print(f'URL: {tunnel_url}')
    print(f'Password: {password}')
    print('='*70)
    print('\n📋 Run on LOCAL machine:\n')
    print(f'python tools/backtest_executor.py \\')
    print(f'  --tunnel-url "{tunnel_url}" \\')
    print(f'  --password "{password}" \\')
    print(f'  --strategy FreqAIHybridStrategy \\')
    print(f'  --timerange 20250901-20251012 \\')
    print(f'  --pairs BTC/USDT:USDT')
    print('\n' + '='*70)
else:
    print('❌ Could not get tunnel URL')
    print('Check process manually in next cell')

In [ ]:
%%capture
# 6. Clone repo
import os
os.chdir('/content')
!rm -rf freqai-futures-strategy
!git clone -q https://github.com/aminak58/freqai-futures-strategy.git
os.chdir('/content/freqai-futures-strategy')
!pip install -q ta-lib-binary 'freqtrade[freqai]'

In [ ]:
# 7. Verify
import os
print(f'✅ Ready: {os.getcwd()}')
!ls -lh | head -10

In [ ]:
# 8. Optional: Mount Drive for data
from google.colab import drive
import zipfile, os
drive.mount('/content/drive')
zip_path = '/content/drive/MyDrive/FreqAI/binance_data.zip'
if os.path.exists(zip_path):
    print('📦 Extracting data...')
    os.chdir('/content/freqai-futures-strategy')
    os.makedirs('user_data/data', exist_ok=True)
    with zipfile.ZipFile(zip_path) as z:
        z.extractall('user_data/data/')
    print('✅ Data ready')
else:
    print('ℹ️  No data in Drive')

In [ ]:
# 9. Keep alive
import time
from IPython.display import clear_output
print('🔄 Session active - Tunnel running')
print('='*70)
print(f'URL: {tunnel_url}')
print(f'Password: {password}')
print('='*70)
print('Updates every 5min\n')
c = 0
while True:
    try:
        time.sleep(300)
        c += 1
        clear_output(wait=True)
        print(f'🔄 Active #{c} ({c*5}min)')
        print('='*70)
        print(f'URL: {tunnel_url}')
        print(f'Password: {password}')
        print('='*70)
        print(time.strftime('%H:%M:%S'))
    except KeyboardInterrupt:
        break

---
## Troubleshooting

In [ ]:
# Check cloudflared process
!ps aux | grep cloudflared

In [ ]:
# Test SSH locally
!ssh -o StrictHostKeyChecking=no root@localhost -p 22 'echo Connection OK'

In [ ]:
# Manual restart cloudflared
!pkill -9 cloudflared
import time
time.sleep(2)
print('Starting fresh cloudflared...')
!/content/cloudflared tunnel --url tcp://localhost:22